In [1]:
import cv2

# Remplacez 'your_image.jpg' par le chemin réel de l'image
image_path = "./images_test.webp"

# Chargez le modèle HOG + SVM pré-entraîné pour la détection des piétons
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Lisez l'image
frame = cv2.imread(image_path)

# Utilisez une image en niveaux de gris, également pour une détection plus rapide
frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Appliquez un flou gaussien à l'image
frame_blur = cv2.GaussianBlur(frame_gray, (3, 3), 0)

# Détectez les piétons dans l'image
boxes, weights = hog.detectMultiScale(frame_blur, winStride=(4, 1), padding=(14, 10), scale=1.05)

# Dessinez des rectangles autour des piétons détectés
for (x, y, w, h) in boxes:
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Affichez le nombre de personnes détectées dans l'image
cv2.putText(frame, f'Persons: {len(boxes)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

# Enregistrez l'image avec les rectangles
output_path = "./image_output.jpg"
cv2.imwrite(output_path, frame)

# Affichez l'image avec les rectangles dessinés
cv2.imshow('Pedestrian Detection', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(f"Image enregistrée avec les rectangles à l'emplacement : {output_path}")


Image enregistrée avec les rectangles à l'emplacement : ./image_output.jpg


In [4]:
import cv2

# Replace 'your_video.mp4' with the actual video path
video_path = "Sil_f.mp4"

# Load the pre-trained HOG + SVM model for pedestrian detection
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Open the video file as a VideoCapture object
cap = cv2.VideoCapture(video_path)

# Check if the video file was successfully opened
if not cap.isOpened():
    print("Could not open video")
    exit()

# Get the original frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Set the output video writer
fourcc = cv2.VideoWriter_fourcc(*'X264')  # You can also try 'X264'
out = cv2.VideoWriter('output_Video1.mp4', fourcc, 20.0, (frame_width, frame_height))


# Initialize consecutive increase and decrease counts
# Variable to count the number of persons
previous_count = 0
consecutive_increase_count = 0
consecutive_decrease_count = 0
increase_delay = 2  # Delay for consecutive increases
decrease_delay = 6  # Delay for consecutive decreases

nbr_persons = []

# Variable to count the number of persons
nbr_rectangles = 0

# Set the number of frames to skip
skip_frames = 6  # You can adjust this value based on your needs

frame_count = 0



while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the frame was successfully read
    if not ret:
        print("End of video.")
        break

    # Increment the frame counter
    frame_count += 1

    # Skip frames if needed
    if frame_count % skip_frames != 0:
        continue

    # Using a greyscale picture, also for faster detection
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to the frame
    frame_blur = cv2.GaussianBlur(frame_gray, (3, 3), 0)

    # Detect pedestrians in the frame
    boxes, weights = hog.detectMultiScale(frame_blur, winStride=(4, 1), padding=(14, 10), scale=1.05)

    # Draw rectangles around detected pedestrians
    for (x, y, w, h) in boxes:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    nbr_persons.append(len(boxes))
    
    cv2.putText(frame, 'Exact match', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Persons : {len(boxes)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    current_count = len(boxes)
    
    
    # Check for consecutive increases or decreases
    if current_count > previous_count:
        consecutive_increase_count += 1
        consecutive_decrease_count = 0
    elif current_count < previous_count:
        consecutive_increase_count = 0
        consecutive_decrease_count += 1
    else:
        consecutive_increase_count = 0
        consecutive_decrease_count = 0

    # Display the number of persons only if there is a change after the delay
    if consecutive_increase_count >= increase_delay or consecutive_decrease_count >= decrease_delay:
        # Display the number of persons detected in the current frame
        cv2.putText(frame, f'Persons: {current_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        previous_count = current_count


    # Display the number of persons only if there is a change after the delay
    # Display the number of persons detected in the current frame
    cv2.putText(frame, 'Frequency Match', (frame_width - 280, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Persons: {previous_count}', (frame_width - 190, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


    out.write(frame)

    # Display the processed frame or perform other tasks
    cv2.imshow('Pedestrian Detection', frame)
    
    

    # Break the loop if the user presses the 'q' key
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the VideoCapture object
cap.release()
out.release()
cv2.destroyAllWindows()

print("Number of persons:", nbr_persons)


In [ ]:
import cv2

# Remplacez 'your_video.mp4' par le chemin réel de la vidéo
video_path = "test_case.mov"

# Charger le modèle HOG + SVM pré-entraîné pour la détection des piétons
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Ouvrir le fichier vidéo en tant qu'objet VideoCapture
cap = cv2.VideoCapture(video_path)

# Vérifier si le fichier vidéo a été ouvert avec succès
if not cap.isOpened():
    print("Impossible d'ouvrir la vidéo")
    exit()

# Obtenir la largeur et la hauteur originales du cadre
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Définir l'écrivain vidéo de sortie
fourcc = cv2.VideoWriter_fourcc(*'X264')  # Vous pouvez également essayer 'X264'
out = cv2.VideoWriter('output_Video2.mp4', fourcc, 20.0, (frame_width, frame_height))

# Initialiser les compteurs d'augmentation et de diminution consécutives
# Variable pour compter le nombre de personnes
previous_count = 0
consecutive_increase_count = 0
consecutive_decrease_count = 0
increase_delay = 2  # Délai pour les augmentations consécutives
decrease_delay = 6  # Délai pour les diminutions consécutives

nbr_persons = []

# Définir le nombre d'images à sauter
skip_frames = 6  # Vous pouvez ajuster cette valeur en fonction de vos besoins

frame_count = 0

while cap.isOpened():
    # Lire une image de la vidéo
    ret, frame = cap.read()

    # Vérifier si l'image a été lue avec succès
    if not ret:
        print("Fin de la vidéo.")
        break

    # Incrémenter le compteur d'images
    frame_count += 1

    # Sauter des images si nécessaire
    if frame_count % skip_frames != 0:
        continue

    # Redimensionner l'image à une nouvelle largeur et hauteur (par exemple, 640x480)
    new_width, new_height = 640, 480
    frame = cv2.resize(frame, (new_width, new_height))

    # Utilisation d'une image en niveaux de gris, également pour une détection plus rapide
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Appliquer un flou gaussien à l'image
    frame_blur = cv2.GaussianBlur(frame_gray, (3, 3), 0)

    # Détecter les piétons dans l'image
    boxes, _ = hog.detectMultiScale(frame_blur, winStride=(8, 3), padding=(4, 4), scale=1.05)

    # Dessiner des rectangles autour des piétons détectés
    for (x, y, w, h) in boxes:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    nbr_persons.append(len(boxes))
    
    # Afficher le texte "Exact match" dans le coin supérieur gauche
    cv2.putText(frame, 'Exact match', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Afficher le nombre de personnes détectées dans le coin supérieur gauche
    cv2.putText(frame, f'Persons: {len(boxes)}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    current_count = len(boxes)
    
    # Vérifier les augmentations ou diminutions consécutives
    if current_count > previous_count:
        consecutive_increase_count += 1
        consecutive_decrease_count = 0
    elif current_count < previous_count:
        consecutive_increase_count = 0
        consecutive_decrease_count += 1
    else:
        consecutive_increase_count = 0
        consecutive_decrease_count = 0

    # Afficher le nombre de personnes uniquement s'il y a un changement après le délai
    if consecutive_increase_count >= increase_delay or consecutive_decrease_count >= decrease_delay:
        # Afficher le nombre de personnes détectées dans l'image actuelle
        cv2.putText(frame, f'Persons: {current_count}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        previous_count = current_count

    # Afficher le texte "Frequency Match" dans le coin supérieur droit
    cv2.putText(frame, 'Frequency Match', (new_width - 280, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Afficher le nombre de personnes détectées dans le coin supérieur droit
    cv2.putText(frame, f'Persons: {previous_count}', (new_width - 280, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    out.write(frame)

    # Afficher le cadre traité ou effectuer d'autres tâches
    cv2.imshow('Détection de piétons', frame)
    
    # Arrêter la boucle si l'utilisateur appuie sur la touche 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Libérer l'objet VideoCapture
cap.release()
out.release()
cv2.destroyAllWindows()

print("Nombre de personnes:", nbr_persons)


In [57]:
import math

len(nbr_persons) # pour 51 secondes 
# pour les 11 premières secondes, on a 1 personnes
# pour les 21 secondes suivantes on a 2 personnes
# pour les 19 secondes suivantes on a 1 personne

nbr_first_frames = len(nbr_persons)*11/51
print(nbr_first_frames)
nbr_middle_frames = math.ceil(len(nbr_persons)*21/51)
print(nbr_middle_frames)
nbr_last_frames = len(nbr_persons)*19/51
print(nbr_last_frames)

nbr_persons_reality = [1]*int(nbr_first_frames) + [2]*int(nbr_middle_frames) + [1]*int(nbr_last_frames)
len(nbr_persons_reality)

percentages = {}

for value in list(range(max(nbr_persons_reality) + 1)):
    percentages[value] = nbr_persons_reality.count(value)/len(nbr_persons_reality)*100

percentages

33.0
63
57.0


{0: 0.0, 1: 58.82352941176471, 2: 41.17647058823529}

In [5]:
import numpy as np

def generate_new_list(original_list, new_length):
    # Calculate the distribution of the original list
    original_distribution = [original_list.count(i) / len(original_list) for i in set(original_list)]

    # Create a modified distribution for the extended length
    modified_distribution = [original_list.count(i) / len(original_list) for i in set(original_list)]

    # Adjust the modified distribution as needed
    # For example, you can increase or decrease the probabilities based on your requirements
    # Here, we simply increase the probability of the last element
    modified_distribution[-1] += 0.1

    # Linearly interpolate between the original and modified distributions
    interpolation_factor = np.linspace(0, 1, new_length)
    combined_distribution = (1 - interpolation_factor)[:, None] * np.array(original_distribution) + \
                            interpolation_factor[:, None] * np.array(modified_distribution)

    # Normalize the combined distribution to ensure probabilities sum to 1
    combined_distribution /= combined_distribution.sum(axis=1)[:, None]

    # Generate a new list based on the combined distribution
    new_list = np.random.choice(list(set(original_list)), size=new_length, p=combined_distribution[-1])

    return new_list.tolist()

# Dans le cas ou on aurait différents backgrounds 
# un apprentissage supervisé aurait été nécessaire 
# pour prouvoir optimiser les paramètres de la fonction generate_new_list
# winstride, padding, scale, etc...



# Set the desired length for the new list (e.g., increase the size gradually)
max_length = 10000
step_size = 100

all_percentages = []

for new_length in range(step_size, max_length + step_size, step_size):
    # Generate a new list with a mixed distribution
    new_list = generate_new_list(nbr_persons, new_length)

    # Convert new_list to a numpy array
    nbr_persons_ = np.array(new_list)

    # Frequency of each value in nbr_persons
    unique_elements, counts_elements = np.unique(nbr_persons_, return_counts=True)

    # Calculate percentages
    counts_elements_percentage = (counts_elements / sum(counts_elements)) * 100

    # Print unique elements and their corresponding percentages
    print(f"Length: {new_length}")
    for value, percentage in zip(unique_elements, counts_elements_percentage):
        print(f"Value: {value}, Percentage: {percentage}%")
    print()

    # Store percentages in the list
    all_percentages.append(counts_elements_percentage)

# Calculate the average of each percentage at the end
average_percentages = np.average(all_percentages, axis=0)
average_percentage_round = np.sort(np.round(average_percentages),axis=-1)

# Print the average percentages
print("Average Percentages:")
for value, average_percentage in zip(unique_elements, average_percentages):
    print(f"Value: {value}, Average Percentage: {average_percentage}%")

for value, average_percentage in zip(unique_elements, average_percentage_round):
    print(f"Value: {value}, Average Percentage (rounded): {average_percentage}%")
    
    
nbr_persons = np.array(nbr_persons)
# frequence de chaque valeur de nbr_persons
unique_elements, counts_elements = np.unique(nbr_persons, return_counts=True)
unique_elements, counts_elements

counts_elements_percentage = counts_elements*100 /sum(counts_elements)
unique_elements,counts_elements_percentage

percentages = dict(zip(unique_elements, counts_elements_percentage))

max(percentages, key=percentages.get)
max(percentages.values())
# take the key if the value is 
personnes_TT = max(((key, value) for key, value in percentages.items() if value > average_percentage_round[len(average_percentage_round)-1]), default=None)
personnes_PT = max(((key,value) for key, value in percentages.items() if value < average_percentage_round[len(average_percentage_round)-1] and value > average_percentage_round[0]), default=None)
personnes_Erreur = min(((key,value) for key, value in percentages.items() if value < average_percentage_round[0]), default=None)

personnes_TT, personnes_PT, personnes_Erreur

print('Dans la vidéo, on a majoritairement ',personnes_TT[0],' personnes')
print('Dans la vidéo, on a pendant quelques temps',personnes_PT[0],' personnes')


Length: 100
Value: 0, Percentage: 11.0%
Value: 1, Percentage: 60.0%
Value: 2, Percentage: 28.999999999999996%

Length: 200
Value: 0, Percentage: 8.0%
Value: 1, Percentage: 59.0%
Value: 2, Percentage: 33.0%

Length: 300
Value: 0, Percentage: 10.666666666666668%
Value: 1, Percentage: 61.66666666666667%
Value: 2, Percentage: 27.666666666666668%

Length: 400
Value: 0, Percentage: 9.75%
Value: 1, Percentage: 62.0%
Value: 2, Percentage: 28.249999999999996%

Length: 500
Value: 0, Percentage: 10.4%
Value: 1, Percentage: 59.599999999999994%
Value: 2, Percentage: 30.0%

Length: 600
Value: 0, Percentage: 11.833333333333334%
Value: 1, Percentage: 60.5%
Value: 2, Percentage: 27.666666666666668%

Length: 700
Value: 0, Percentage: 11.857142857142858%
Value: 1, Percentage: 59.14285714285714%
Value: 2, Percentage: 28.999999999999996%

Length: 800
Value: 0, Percentage: 9.75%
Value: 1, Percentage: 62.25000000000001%
Value: 2, Percentage: 28.000000000000004%

Length: 900
Value: 0, Percentage: 10.444444444

In [52]:
percentage = average_percentage_round[1]
average_percentage_round[1] = average_percentage_round[2]
average_percentage_round[2] = percentage
average_percentage_round

array([11., 61., 28.])

In [62]:
# difference de pourcentage entre average percentage round et les pourcentages de la vidéo
# je veux avoir un pourcentage en forme d'accuracy 
# loss 


diff = []
for i in range(len(average_percentage_round)):
    diff.append(abs(average_percentage_round[i] - percentages[i]))
diff


[11.0, 2.17647058823529, 13.17647058823529]

In [ ]:
# Supposons que vous ayez défini un seuil de tolérance, par exemple 5%.
seuil_tolerance = 5

# Calculez le nombre de prédictions correctes en fonction du seuil de tolérance.
predictions_correctes = sum(diff[i] <= seuil_tolerance for i in range(len(diff)))

# Calculez l'accuracy en pourcentage.
accuracy = (predictions_correctes / len(diff)) * 100

print(f"Accuracy: {accuracy:.2f}%")


In [49]:
percentages

{0: 11.764705882352942, 1: 67.3202614379085, 2: 20.915032679738562}

In [25]:
# accuracy 


def exact_match(nbr_persons,nbr_persons_detected):
    
    exact_match = 0 
    for i in range(len(nbr_persons)) :
        
        person_count_detected = nbr_persons_detected[i]
        person_count = nbr_persons[i]
        
        # print("person count : ", person_count)
        # print("person count detected : ", person_count_detected)
        
        if person_count == person_count_detected:
            exact_match += 1
    

    print("accuracy is : ", (exact_match / len(nbr_persons)) * 100)
    print("exact match : ", exact_match)
    
    return (exact_match / len(nbr_persons)) * 100 

exact_match(nbr_persons_reality,nbr_persons)
    

accuracy is :  71.05263157894737
exact match :  54


71.05263157894737